<a href="https://colab.research.google.com/github/abosedealli/Stockprediction/blob/main/LSTM_open%2C_low_and_high_prediction_date_from_2002_till_date.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LSTM open, low and high prediction date from 2002 till date

In [24]:
!pip install pandas
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

# load your dataset
# load your dataset
start = "1900-01-01"
data = yf.download('NFLX', start=start)

# create a dataframe with only the open, low, and high prices
data = data[['Open', 'Low', 'High']]
prices = data

# create a variable to store the number of days you want to forecast
forecast_days = 30


[*********************100%***********************]  1 of 1 completed


In [26]:
data

,Open,Low,High
Date,,,
2002-05-23 00:00:00-04:00,1.156429,1.145714,1.242857
2002-05-24 00:00:00-04:00,1.214286,1.197143,1.225000
2002-05-28 00:00:00-04:00,1.213571,1.157143,1.232143
2002-05-29 00:00:00-04:00,1.164286,1.085714,1.164286
2002-05-30 00:00:00-04:00,1.107857,1.071429,1.107857
...,...,...,...
2023-01-09 00:00:00-05:00,316.829987,313.220001,321.700012
2023-01-10 00:00:00-05:00,311.070007,311.070007,329.350006
2023-01-11 00:00:00-05:00,326.500000,321.350006,328.880005


In [12]:
# create a variable to store the percentage of data you want to use for testing
test_data_size = 0.2

# create a variable to store the number of days in your dataset
days_in_dataset = len(data)

# create variables to store the number of days you want to use for testing and training
test_days = int(days_in_dataset * test_data_size)
train_days = days_in_dataset - test_days

# create a training dataset
train_data = data.iloc[:train_days]

# create a testing dataset
test_data = data.iloc[train_days:]

In [13]:
# create a scaler to normalize the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# fit the scaler to the training data
scaler.fit(train_data)

# normalize the training data
train_data = scaler.transform(train_data)

# create a dataset with only the open, low, and high prices for the last forecast_days days
X_train = []
y_train = []
for i in range(forecast_days, train_days):
    X_train.append(train_data[i-forecast_days:i])
    y_train.append(train_data[i])
X_train, y_train = np.array(X_train), np.array(y_train)

# reshape the data to be 3D
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 3))

# create a LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 3)))
model.add(LSTM(50))
model.add(Dense(3))

# compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# fit the model to the training data
model.fit(X_train, y_train, epochs=100, batch_size=32)


Epoch 1/100
130/130 [==============================] - 8s 33ms/step - loss: 0.0024
Epoch 2/100
130/130 [==============================] - 4s 33ms/step - loss: 2.7351e-04
Epoch 3/100
130/130 [==============================] - 4s 33ms/step - loss: 2.4134e-04
Epoch 4/100
130/130 [==============================] - 4s 33ms/step - loss: 2.5863e-04
Epoch 5/100
130/130 [==============================] - 4s 32ms/step - loss: 2.6891e-04
Epoch 6/100
130/130 [==============================] - 4s 33ms/step - loss: 1.7457e-04
Epoch 7/100
130/130 [==============================] - 4s 32ms/step - loss: 1.6384e-04
Epoch 8/100
130/130 [==============================] - 4s 33ms/step - loss: 1.7716e-04
Epoch 9/100
130/130 [==============================] - 4s 33ms/step - loss: 1.7068e-04
Epoch 10/100
130/130 [==============================] - 4s 33ms/step - loss: 1.5946e-04
Epoch 11/100
130/130 [==============================] - 4s 33ms/step - loss: 1.3178e-04
Epoch 12/100
130/130 [=======================

In [14]:
# normalize the testing data
test_data = scaler.transform(test_data)

In [15]:
# create a dataset with only the open, low, and high prices for the last forecast_days days
X_test = []
y_test = []
for i in range(forecast_days, test_days):
    X_test.append(test_data[i-forecast_days:i])
    y_test.append(test_data[i])
X_test, y_test = np.array(X_test), np.array(y_test)

# reshape the data to be 3D
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 3))

# make predictions
predictions = model.predict(X_test)

# inverse transform the predictions to get the original prices
predictions = scaler.inverse_transform(predictions)
y_test = scaler.inverse_transform(y_test)

# calculate the mean squared error of the predictions
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)
ape = np.abs((y_test - predictions) / y_test)
# Calculate the mean of the absolute percentage difference
mape = np.mean(ape) * 100
print("MAPE: ", mape)



32/32 [==============================] - 1s 11ms/step
Mean Squared Error: 256.71656000334787
MAPE:  2.3667153320215193


In [23]:
data

[Scatter({
     'name': 'Actual Open Prices',
     'x': array([   0,    1,    2, ..., 1006, 1007, 1008]),
     'y': array([334.23999023, 349.6000061 , 354.        , ..., 326.5       ,
                 332.5       , 329.97000122])
 }), Scatter({
     'name': 'Predicted Open Prices',
     'x': array([   0,    1,    2, ..., 1006, 1007, 1008]),
     'y': array([337.61542, 328.24976, 358.28873, ..., 327.57462, 321.7805 , 328.36417],
                dtype=float32)
 }), Scatter({
     'name': 'Forecasted Open Prices',
     'x': array([1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019,
                 1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031,
                 1032, 1033, 1034, 1035, 1036, 1037, 1038]),
     'y': array([328.36417, 527.1168 , 472.99667, 477.21893, 473.2993 , 512.8273 ,
                 487.29224, 472.52255, 469.47125, 466.13574, 481.34302, 467.14563,
                 484.31537, 485.7151 , 454.96042, 445.93753, 384.64935, 373.89

In [17]:
import plotly.graph_objs as go

# generate new data
X_forecast = []
for i in range(forecast_days, forecast_days+30):
    X_forecast.append(prices[i-forecast_days:i])
X_forecast = np.array(X_forecast)


forecast = model.predict(X_forecast)

1/1 [==============================] - 0s 28ms/step


In [18]:
# Get the last predicted value
last_prediction = predictions[-1]

# Add a dimension to the last value of predictions
last_prediction = np.expand_dims(last_prediction, axis=0)
forecast = scaler.inverse_transform(forecast)
# Insert the last value of predictions at the 0th index of forecast
forecast = np.insert(forecast, 0, last_prediction, axis=0)
trace1 = go.Scatter(x=np.arange(y_test.shape[0]), y=y_test[:,0], name='Actual Open Prices')
trace2 = go.Scatter(x=np.arange(predictions.shape[0]), y=predictions[:,0], name='Predicted Open Prices')
trace3 = go.Scatter(x=np.arange(predictions.shape[0]-1, predictions.shape[0] + forecast.shape[0]-1), y=forecast[:,0], name='Forecasted Open Prices')
trace4 = go.Scatter(x=np.arange(y_test.shape[0]), y=y_test[:,1], name='Actual Low Prices')
trace5 = go.Scatter(x=np.arange(predictions.shape[0]), y=predictions[:,1], name='Predicted Low Prices')
trace6 = go.Scatter(x=np.arange(predictions.shape[0]-1, predictions.shape[0] + forecast.shape[0]-1), y=forecast[:,1], name='Forecasted Low Prices')
trace7 = go.Scatter(x=np.arange(y_test.shape[0]), y=y_test[:,2], name='Actual High Prices')
trace8 = go.Scatter(x=np.arange(predictions.shape[0]), y=predictions[:,2], name='Predicted High Prices')
trace9 = go.Scatter(x=np.arange(predictions.shape[0]-1, predictions.shape[0] + forecast.shape[0]-1), y=forecast[:,2], name='Forecasted High Prices')

data = [trace1, trace2, trace3, trace4, trace5, trace6, trace7, trace8, trace9]
layout = go.Layout(title='Actual, Predicted and Forecasted Prices', xaxis=dict(title='Time'), yaxis=dict(title='Price'))
fig = go.Figure(data=data, layout=layout)
fig.show()

     

In [19]:
trace1 = go.Scatter(x=np.arange(y_test.shape[0]), y=y_test[:,0], name='Actual Open Prices')
trace2 = go.Scatter(x=np.arange(predictions.shape[0]), y=predictions[:,0], name='Predicted Open Prices')
trace3 = go.Scatter(x=np.arange(predictions.shape[0]-1, predictions.shape[0] + forecast.shape[0]-1), y=forecast[:,0], name='Forecasted Open Prices')
trace4 = go.Scatter(x=np.arange(y_test.shape[0]), y=y_test[:,1], name='Actual Low Prices')
trace5 = go.Scatter(x=np.arange(predictions.shape[0]), y=predictions[:,1], name='Predicted Low Prices')
trace6 = go.Scatter(x=np.arange(predictions.shape[0]-1, predictions.shape[0] + forecast.shape[0]-1), y=forecast[:,1], name='Forecasted Low Prices')
trace7 = go.Scatter(x=np.arange(y_test.shape[0]), y=y_test[:,2], name='Actual High Prices')
trace8 = go.Scatter(x=np.arange(predictions.shape[0]), y=predictions[:,2], name='Predicted High Prices')
trace9 = go.Scatter(x=np.arange(predictions.shape[0]-1, predictions.shape[0] + forecast.shape[0]-1), y=forecast[:,2], name='Forecasted High Prices')

data = [trace1, trace2, trace3, trace4, trace5, trace6, trace7, trace8, trace9]
layout = go.Layout(title='Actual, Predicted and Forecasted Prices', xaxis=dict(title='Time'), yaxis=dict(title='Price'))
fig = go.Figure(data=data, layout=layout)

fig.update_layout(
    title="Netflix Price Prediction",
    xaxis_title="Days",
    yaxis_title="Price",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="Black"
    )
)
     
